Vector Search Qdrant

## 1. Importación de librerías y conexión con Qdrant

Primero asegúrate de correr e instalar las dependencias necesarias
Y dejar el contenedor corriendo. <p>
Setup: [set_up_qdrant]()

Importación de librerías
- Clase `QdrantClient`: permite establecer conexión con el servicio de Qdrant.
- Módulo `models`: permite establecer configuraciones y parámetros necesarios.

In [4]:
from qdrant_client import QdrantClient, models

In [5]:
# Inicializamos el cliente
client = QdrantClient("http://localhost:6333")

## 2. 
